In [ ]:
# !pip install --user matplotlib seaborn

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [1]:
from bert.model import load_model_from_checkpoint

model = load_model_from_checkpoint('/scratch/pstjohn/dist_sl128_nowd_checkpoints/ckpt.h5')
model.summary()

TypeError: 'dict' object is not callable

In [ ]:
tf.keras.models.save_model

In [ ]:
tf.keras.models.save_model(model, '/scratch/pstjohn/dist_sl128_nowd_checkpoints/tfckpt', save_format='tf')

In [ ]:
model.save('/scratch/pstjohn/dist_sl128_nowd_checkpoints/tfckpt')

In [ ]:
tf.saved_model.save(model, '/scratch/pstjohn/dist_sl128_nowd_checkpoints/tfckpt')

In [ ]:
model = tf.keras.models.load_model('/scratch/pstjohn/dist_sl128_nowd_checkpoints/tfckpt')

In [ ]:
from bert.dataset import create_masked_input_dataset

valid_data = create_masked_input_dataset(
    sequence_path='../uniparc_data/dev_uniref50.txt.gz',
    max_sequence_length=128,
    batch_size=256)

In [ ]:
masked_seq, true_values = next(iter(valid_data))

In [ ]:
from bert.losses import ECE
predictions = tf.nn.softmax(model.predict(masked_seq))
ECE(true_values, predictions)

In [ ]:
masked_entries = tf.not_equal(true_values, 0)
y_true_mask = tf.boolean_mask(true_values, masked_entries)
y_logit_mask = tf.boolean_mask(predictions, masked_entries)
y_pred_mask = y_logit_mask.numpy().argmax(-1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_true_mask.numpy(), y_pred_mask)

In [ ]:
y_true_mask.numpy().shape

In [ ]:
aas = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
confusion_mat = pd.DataFrame([y_pred_mask, y_true_mask.numpy()], index=['Pred', 'True']
                            ).T.groupby(['Pred', 'True']).size().reset_index().pivot(
    index='True', columns='Pred', values=0).fillna(0)

confusion_mat.index = aas
confusion_mat.columns = aas

In [ ]:
height = 8
ratio = 5

# Set up the subplot grid
f = plt.figure(figsize=(height, height))
gs = plt.GridSpec(ratio + 1, ratio + 1)

ax_joint = f.add_subplot(gs[1:, :-1])
ax_marg_x = f.add_subplot(gs[0, :-1], sharex=ax_joint)
ax_marg_y = f.add_subplot(gs[1:, -1], sharey=ax_joint)

fig = f
ax_joint = ax_joint
ax_marg_x = ax_marg_x
ax_marg_y = ax_marg_y

# Turn off tick visibility for the measure axis on the marginal plots
plt.setp(ax_marg_x.get_xticklabels(), visible=False)
plt.setp(ax_marg_y.get_yticklabels(), visible=False)

# Turn off the ticks on the density axis for the marginal plots
plt.setp(ax_marg_x.yaxis.get_majorticklines(), visible=False)
plt.setp(ax_marg_x.yaxis.get_minorticklines(), visible=False)
plt.setp(ax_marg_y.xaxis.get_majorticklines(), visible=False)
plt.setp(ax_marg_y.xaxis.get_minorticklines(), visible=False)
plt.setp(ax_marg_x.get_yticklabels(), visible=False)
plt.setp(ax_marg_y.get_xticklabels(), visible=False)
ax_marg_x.yaxis.grid(False)
ax_marg_y.xaxis.grid(False)

sns.heatmap(confusion_mat, ax=ax_joint, cbar=False, cmap='Blues')
_ = ax_marg_x.bar(aas, height=confusion_mat.sum(0), width=1., align='edge', facecolor='.5')
_ = ax_marg_y.barh(aas, width=confusion_mat.sum(1), height=1., align='edge', facecolor='.5')

sns.despine(ax=ax_marg_x, left=True)
sns.despine(ax=ax_marg_y, bottom=True)

_ = ax_joint.set_xticks(np.arange(20) + 0.5)
_ = ax_joint.set_xticklabels(aas)
_ = ax_joint.set_yticks(np.arange(20) + 0.5)
_ = ax_joint.set_yticklabels(aas)

In [ ]:
inputs = tf.keras.Model(model.layers[1].input, model.layers[1].output)(masked_seq)
inputs._keras_mask

In [ ]:
transformer = model.layers[2]

In [ ]:
self = transformer.attention_layer
mask = inputs._keras_mask
training=True

from bert.attention_utils import create_attention_mask, relative_attention_inner, initializer

In [ ]:
input_shape = tf.shape(inputs) # [B, S, N*H]

# query and key can be the same vector
qk = self.transpose_scores(self.qk_layer(inputs))  # [B,N,S,H]
value = self.transpose_scores(self.value_layer(inputs))  # [B,N,S,H]

# Equation 1 of "Attention is all you need"
attention_scores = self.calculate_attention(qk, input_shape)  # [B,N,S,S]

In [ ]:
inputs.shape

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(self.relations_keys_embedding.weights[0].numpy())

In [ ]:
# zero out masked values
attention_mask = self.create_attention_mask(input_shape, mask)
attention_scores = attention_scores * 1000 + (1. - attention_mask) * -10000.0
attention_probs = tf.nn.softmax(attention_scores)  # [B,N,S,S]

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(attention_probs[2, 0].numpy())

In [ ]:
context_layer = tf.matmul(attention_probs, value)  # [B,N,S,S]

In [ ]:
context_layer.shape

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(context_layer[2, 0].numpy())

In [ ]:
context_layer = tf.transpose(a=context_layer, perm=[0, 2, 1, 3])            

In [ ]:
context_layer.shape

In [ ]:
output_shape = [input_shape[0], input_shape[1], self.num_heads*self.units]
context_layer = tf.reshape(context_layer, output_shape)

In [ ]:
context_layer.shape

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(context_layer[0, :, :10].numpy())

In [ ]:
mask = inputs._keras_mask
attention_mask = self.create_attention_mask(mask)
attention_scores_mask = attention_scores + (1. - attention_mask) * -10000.0
attention_probs = tf.nn.softmax(attention_scores_mask)  # [B,N,S,S]
context_layer = tf.matmul(attention_probs, value)  # [B,N,S,S]

input_shape  = tf.shape(inputs)
output_shape = [input_shape[0], input_shape[1], self.num_heads*self.units]
# context_layer = tf.reshape(context_layer, output_shape)

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(attention_mask[3, 0].numpy())

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(attention_scores[3, 0].numpy())

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(attention_scores_mask[3, 0].numpy())

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(attention_probs[3, 0].numpy())

In [ ]:
context_layer.shape

In [ ]:
context_layer.shape

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(tf.reshape(tf.transpose(a=context_layer, perm=[0, 2, 1, 3]), input_shape)[3].numpy())